In [213]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys

from tqdm import tqdm
from fake_useragent import UserAgent

# Парсим обращения жителей к главе города Троицк (Москва)

In [14]:
main_link="http://appeal.admtroitsk.ru/viewforum.php?f=9"

In [41]:
root_link="http://appeal.admtroitsk.ru"

### Интересующие аттрибуты:

In [125]:
col_names=["link","date","views", "author", "title", "text"]

# Парсинг:

In [15]:
#&mode=&start=0
def page_n(n):
    return "&mode=&start="+str(n*10)

### Номер последней страницы обращений к главе города - 1000 (16.05.2017)
### (всего 1000 с небольшим обращений, по 10 обращений на странице)

In [171]:
page_link=main_link+page_n(5)

In [172]:
def get_soup(page_link):
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [173]:
soup = get_soup(page_link)

In [174]:
item_data = soup.findAll("div", attrs={'class':'white-bg nopadding'})

In [175]:
len(item_data)

10

In [176]:
item_info=item_data[0]

In [177]:
item_info

<div class="white-bg nopadding">
<div class="forumrow_container topicrow_container">
<div class="icon" id="icon5284" rel="9">
<a class="tooltipped" data-tooltip="Перейти к последнему сообщению" href="./viewtopic.php?f=9&amp;t=5284&amp;p=17025#p17025"><i aria-hidden="true" class="fa fa-comment-o"></i></a>
</div>
<div class="topic_author"><a class="username" href="./memberlist.php?mode=viewprofile&amp;u=1451">tvantenna</a></div>
<a class="forumrow topicrow white-bg answered_topic" href="./viewtopic.php?f=9&amp;t=5284">
<h3>Случай с короновирусом.Никому не нужны</h3>
<i aria-hidden="true" class="fa fa-clock-o"></i> 22.04.2020 16:40 <i aria-hidden="true" class="fa fa-calendar-check-o"></i> 9 <i aria-hidden="true" class="fa fa-eye"></i> 1100
			<!-- topic_read_locked Эта тема закрыта, Вы не можете редактировать и оставлять сообщения в ней.<br> -->
</a>
</div>
</div>

In [74]:
def get_item_ref(item_info, rt_link="http://appeal.admtroitsk.ru"):
    item_ref=item_info.findAll("a",attrs={'class':'tooltipped'})
    rel_ref=None if not item_ref else item_ref[0]['href'][1:]
    item_link=None if not rel_ref else rt_link+rel_ref
    return item_link

In [76]:
def get_item_author(item_info):
    auth_l=item_info.findAll("a",attrs={'class':'username'})
    auth=None if not auth_l else auth_l[0].text
    return auth

In [80]:
def get_header(item_info):
    hd=item_info.find('h3')
    header=None if not hd else hd.text.replace('.','')
    return header

In [185]:
def get_text_date(ref):
    item_cont=get_soup(ref)
    text_data=item_cont.find("div", attrs={'class':'message'})
    text=None if not text_data else text_data.text
    date=item_cont.find("p", attrs={'class':'date'})
    date_time=None if not date else date.text
    return text, date_time

In [199]:
def get_views(item_info):
    date=item_info.find('a', attrs={'class':'forumrow topicrow white-bg answered_topic'})
    if not date:
        return None
    else:
        dd=date.text.strip()
    tokens=dd.split()
    views=tokens[-1]
    return views

In [187]:
def get_data_line(item_info):
    ref=get_item_ref(item_info)
    author=get_item_author(item_info)
    views=get_views(item_info)
    header=get_header(item_info)
    text,date=get_text_date(ref)
    #header, date, views=get_aux_data(item_info)
    return {'link':ref,'date':date,'views':views,'author':author,'title':header,'text':text}

In [214]:
Dudochkin_data_v2=pd.DataFrame(columns=col_names)
for i in tqdm(range(0,101)):
    page_link=main_link+page_n(i)
    soup = get_soup(page_link)
    item_data = soup.findAll("div", attrs={'class':'white-bg nopadding'})
    time.sleep(0.1)
    for item_info in item_data:
        Dudochkin_data_v2=Dudochkin_data_v2.append(get_data_line(item_info),ignore_index=True)
        time.sleep(0.1)

100%|████████████████████████████████████████████████████████████████████████████████| 101/101 [18:41<00:00, 11.11s/it]


In [215]:
Dudochkin_data_v2.to_csv("Dudochkin_queries_v2.csv", header=Dudochkin_data_v2.columns)

In [216]:
Dudochkin_data_v2.head()

,link,date,views,author,title,text
0,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,Сегодня 13:39,None,Lappusyonok,Устраните последствия непогоды,"Здравствуйте, на крыше дома по адресу микрорай..."
1,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,Сегодня 00:23,None,Anton1983,Площадка на ул Солнечной 4/6 Центральная 30,"Поймите, как жителей людей, чьи окна выходят в..."
2,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,12.06.2020 20:03,None,iv1970,"Беседка за Октябрьским, 2Терпение жителей на и...",Уважаемый Владимир Евгеньевич! С 2017 года тян...
3,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,12.06.2020 15:21,None,abryss,Борщевик в Троицке - 2,Здравствуйте. Ровно год назад я задавала вопро...
4,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,11.06.2020 01:10,205,Ann,затопление подъезда в доме,"Владимир Евгеньевич, здравствуйте.\nВ доме, по..."


In [227]:
Dudochkin_data_v2.loc[1,'date']=Dudochkin_data_v2.loc[1,'date'].replace("Сегодня", "13.06.2020")
Dudochkin_data_v2.loc[0,'date']=Dudochkin_data_v2.loc[0,'date'].replace("Сегодня", "13.06.2020")

In [228]:
Dudochkin_data_v2.head()

,link,date,views,author,title,text
0,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,13.06.2020 13:39,None,Lappusyonok,Устраните последствия непогоды,"Здравствуйте, на крыше дома по адресу микрорай..."
1,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,13.06.2020 00:23,None,Anton1983,Площадка на ул Солнечной 4/6 Центральная 30,"Поймите, как жителей людей, чьи окна выходят в..."
2,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,12.06.2020 20:03,None,iv1970,"Беседка за Октябрьским, 2Терпение жителей на и...",Уважаемый Владимир Евгеньевич! С 2017 года тян...
3,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,12.06.2020 15:21,None,abryss,Борщевик в Троицке - 2,Здравствуйте. Ровно год назад я задавала вопро...
4,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,11.06.2020 01:10,205,Ann,затопление подъезда в доме,"Владимир Евгеньевич, здравствуйте.\nВ доме, по..."


In [230]:
Dudochkin_data_v2.date=pd.to_datetime(Dudochkin_data_v2.date, errors='coerce')

In [234]:
Dudochkin_data_v2.isna().sum()

link      0
date      0
views     6
author    1
title     0
text      0
dtype: int64

# Полученные данные:

In [232]:
Dudochkin_data_v2.head(10)

,link,date,views,author,title,text
0,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-06-13 13:39:00,None,Lappusyonok,Устраните последствия непогоды,"Здравствуйте, на крыше дома по адресу микрорай..."
1,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-06-13 00:23:00,None,Anton1983,Площадка на ул Солнечной 4/6 Центральная 30,"Поймите, как жителей людей, чьи окна выходят в..."
2,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-12-06 20:03:00,None,iv1970,"Беседка за Октябрьским, 2Терпение жителей на и...",Уважаемый Владимир Евгеньевич! С 2017 года тян...
3,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-12-06 15:21:00,None,abryss,Борщевик в Троицке - 2,Здравствуйте. Ровно год назад я задавала вопро...
4,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-11-06 01:10:00,205,Ann,затопление подъезда в доме,"Владимир Евгеньевич, здравствуйте.\nВ доме, по..."
5,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-11-06 19:50:00,152,Юрий Митрофанович,Детские площадки,Детские площадки в Москве должны быть закрыты ...
6,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-11-06 09:00:00,176,Zilber,Академический сквер,Владимир Евгеньевич добрый день!\nКак же здоро...
7,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-09-06 14:14:00,188,dashenka.ivanova.11,Отказ в выплате на ребенка до 3х лет,"Владимир Евгеньевич, добрый день, \n18 мая под..."
8,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-02-06 23:34:00,366,nataly,спортивная площадка на Юбилейной,Уважаемый Владимир Евгеньевич!\nПриходится опя...
9,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-05-06 00:23:00,None,StepaN,Предложение на перспективу,"Здравствуйте, Владимир Евгеньевич.\nПризываю в..."


In [233]:
Dudochkin_data_v2.to_csv("Dudochkin_queries.csv", header=Dudochkin_data_v2.columns)

In [1]:
import pandas as pd

In [4]:
myDf=pd.read_csv("Dudochkin_queries.csv", index_col=False)

In [7]:
myDf.columns

Index(['Unnamed: 0', 'link', 'date', 'views', 'author', 'title', 'text'], dtype='object')

In [8]:
myDf.drop("Unnamed: 0",axis=1,inplace=True)

In [9]:
myDf.head(10)

,link,date,views,author,title,text
0,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-06-13 13:39:00,NaN,Lappusyonok,Устраните последствия непогоды,"Здравствуйте, на крыше дома по адресу микрорай..."
1,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-06-13 00:23:00,NaN,Anton1983,Площадка на ул Солнечной 4/6 Центральная 30,"Поймите, как жителей людей, чьи окна выходят в..."
2,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-12-06 20:03:00,NaN,iv1970,"Беседка за Октябрьским, 2Терпение жителей на и...",Уважаемый Владимир Евгеньевич! С 2017 года тян...
3,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-12-06 15:21:00,NaN,abryss,Борщевик в Троицке - 2,Здравствуйте. Ровно год назад я задавала вопро...
4,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-11-06 01:10:00,205.0,Ann,затопление подъезда в доме,"Владимир Евгеньевич, здравствуйте.\nВ доме, по..."
5,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-11-06 19:50:00,152.0,Юрий Митрофанович,Детские площадки,Детские площадки в Москве должны быть закрыты ...
6,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-11-06 09:00:00,176.0,Zilber,Академический сквер,Владимир Евгеньевич добрый день!\nКак же здоро...
7,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-09-06 14:14:00,188.0,dashenka.ivanova.11,Отказ в выплате на ребенка до 3х лет,"Владимир Евгеньевич, добрый день, \n18 мая под..."
8,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-02-06 23:34:00,366.0,nataly,спортивная площадка на Юбилейной,Уважаемый Владимир Евгеньевич!\nПриходится опя...
9,http://appeal.admtroitsk.ru/viewtopic.php?f=9&...,2020-05-06 00:23:00,NaN,StepaN,Предложение на перспективу,"Здравствуйте, Владимир Евгеньевич.\nПризываю в..."
